## Reading of ASCII files created for cam diagnostics tool

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os
from helper_funcs import read_file_custom, read_var_info_michaels_excel
VERBOSE = True

### Paths and file definitions

Please change accordingly if you execute this notebook on your local machine.

In [2]:
data_dir = "./data/michael_ascii_read/"
case_ok = "./data/from_ada/table_GLBL_ANN_obs_FIXED.asc"

michaels_excel = data_dir + "obs-comp-noresmversions.xlsx"

### Importing supplementary information from Excel table

Let's begin with reading the variable information from the excel table. Note that this is not strictly required but helps us below to display the results in a more intuitive manner, when analysing the data. The custom method that we use is:

In [3]:
help(read_var_info_michaels_excel)

Help on function read_var_info_michaels_excel in module helper_funcs:

read_var_info_michaels_excel(xlspath)
    Read short description strings for variables
    
    The strings are available for some of the variables in Michaels analysis
    excel table (column 3, sheet "DATA")
    
    Parameters
    ----------
    xlspath : location of excel spreadsheet
    
    Returns
    -------
    dict 
        dictionary containing all variable names (keys) and corresponding
        description strings (if applicable, else empty string)



Load the information.

In [4]:
var_info_dict = read_var_info_michaels_excel(michaels_excel)
var_info_dict

{'AODDUST': '',
 'AODVIS': '',
 'CLDTOT_CLOUDSAT': '',
 'CLDTOT_ISCCP': 'Total cloud cover',
 'FLDS_ISCCP': 'LW down SRF',
 'FLNS_ISCCP': 'LW net SRF',
 'FLNT_CAM': '',
 'FLUTC_CERES': '',
 'FLUTC_CERES-EBAF': 'LW up Top Clearsky',
 'FLUTC_ERBE': '',
 'FLUT_CERES': '',
 'FLUT_CERES-EBAF': 'LW up Top',
 'FLUT_ERBE': '',
 'FSDS_ISCCP': 'SW down SRF',
 'FSNS_ISCCP': 'SW net SRF',
 'FSNS_LARYEA': '',
 'FSNTOAC_CERES': 'SW net TOA clearsky',
 'FSNTOAC_ERBE': '',
 'FSNTOA_CERES': 'SW net TOA',
 'FSNTOA_ERBE': '',
 'FSNT_CAM': '',
 'LHFLX_ERA40': '',
 'LHFLX_JRA25': 'Lat Heat Flux',
 'LHFLX_WHOI': '',
 'LWCF_CERES': '',
 'LWCF_CERES-EBAF': 'LW Cloud Forc',
 'LWCF_ERBE': '',
 'PRECT_GPCP': 'Precipitation',
 'PREH2O_AIRS': '',
 'PREH2O_ERA40': 'Precipitable water',
 'PREH2O_ERAI': '',
 'PREH2O_JRA25': '',
 'PREH2O_NVAP': '',
 'PSL_ERAI': '',
 'PSL_JRA25': 'SeaLev pressure',
 'RESSURF': 'SRF net flux',
 'RESTOA_CERES-EBAF': 'TOA  net flux',
 'RESTOA_ERBE': '',
 'RESTOM': 'TOmodel net flux',
 'SH

### Search and load ASCII files (either using .asc or .webarchive file type)

In [5]:
files = sorted(glob(data_dir + "*.webarchive"))

for file in files:
    print(file)
    
test_file = files[0]

print("TEST FILE: {}".format(os.path.basename(test_file)))

./data/michael_ascii_read/N1850C53CLM45L32_f09_tn11_191017 (yrs 71-100).webarchive
./data/michael_ascii_read/N1850_f09_tn14_230218 (yrs 1-20).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 185-215).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 310-340).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 80-110).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r265_ctrl_20180411 (yrs 90-120).webarchive
TEST FILE: N1850C53CLM45L32_f09_tn11_191017 (yrs 71-100).webarchive


Try read first file as is with pandas

In [6]:
try:
    frame = pd.read_csv(test_file, encoding="latin-1")
except Exception as e:
    print(repr(e))
frame.head()

,bplist00Ñ_WebMainResourceÕ
0,_WebResourceTextEncodingName^WebResourceUR...
1,TEST CASE: N1850C53CLM45L32_f09_tn11_191017 (y...
2,CONTROL CASE: OBS data
3,Variable N1850C53CLM45L32_f09_tn11_191017 ...
4,...


This did not work (it basically did not separate the individual columns). The same is the case for the file that includes a whitespace at the problematic variables.

In [7]:
frame = pd.read_csv(case_ok)
frame.head()
frame.shape

(67, 1)

This did not really work since the data is not splitted by columns but includes one column containing the content of each row. The reading has to be done from scratch, especially also because there is some variables with too long names (e.g. L.25 and L. 28) that stick together the first two columns. 

This folder contains a file ``helper_funcs.py`` in which I defined a custom read function ``read_file_custom`` that can convert these files into pandas dataframes.

In [8]:
help(read_file_custom)

Help on function read_file_custom in module helper_funcs:

read_file_custom(fpath, var_info_dict=None, run_id=None, verbose=False)
    Custom ASCII conversion method 
    
    Parameters
    ----------
    fpath : str
        path to file location
    var_info_dict : dict
        optinal dictionary that contains description strings for each of the 
        variables (e.g. retrieved using :func:`read_var_info_michaels_excel`)
    run_id : str, int or dict
        string or integer that may be used as index specifying the model run 
        and that should be used in the Dataframe for the index specifying the 
        model run (only relevant if multiple files are loaded and concatenated 
        into one dataframe). If None, the "TEST CASE" ID, specified in the 
        file header is used for the index.
    verbose : bool
        if True, print output (defaults to False)
        
    Returns
    -------
    Dataframe 
        pandas data frame ready for further analysis. NOTE: the retu

Now load the first file using this function (without providing the optional parameter *var_info_dict*, i.e. the info from Michael's Excel sheet.

In [9]:
df = read_file_custom(test_file, verbose=VERBOSE)
df

Ignoring line: bplist00Ñ_WebMainResourceÕ	
Ignoring line: 
Ignoring line: 
Ignoring line: _WebResourceTextEncodingName^WebResourceURL_WebResourceFrameName_WebResourceData_WebResourceMIMETypeUUTF-8_http://ns2345k.web.sigma2.no/noresm_diagnostics/N1850C53CLM45L32_f09_tn11_191017/CAM_DIAG/yrs71to100-obs/set1/table_GLBL_ANN_obs.ascPO}<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">DIAG SET 1: ANN MEANS GLOBAL
Ignoring line:  
Ignoring line:  
Ignoring line:  
Problem case FSNTOA_CERES-EBAF
Problem case FSNTOAC_CERES-EBAF
Test case: N1850C53CLM45L32_f09_tn11_191017
Control case: OBS data


Flag  \
Run                              Years  Variable           Description          
N1850C53CLM45L32_f09_tn11_191017 71-100 RESTOM                          False   
                                        RESSURF                         False   
                                        RESTOA_CERES-EBAF               False   
                                        RESTOA_ERBE                     False   
                                        SOLIN_CERES-EBAF                False   
                                        SOLIN_CERES                     False   
                                        CLDTOT_ISCCP                    False   
                                        CLDTOT_CLOUDSAT                 False   
                                        FLDS_ISCCP                      False   
                                        FLNS_ISCCP                      False   
                                        FLUT_CERES-EBAF                 False   
                                        FLUT_CERES                      False   
                                        FLUT_ERBE                       False   
                                        FLUTC_CERES-EBAF                False   
                                        FLUTC_CERES                     False   
                                        FLUTC_ERBE                      False   
                                        FLNT_CAM                        False   
                                        FSDS_ISCCP                      False   
                                        FSNS_ISCCP                      False   
                                        FSNS_LARYEA                     False   
                                        FSNTOA_CERES-EBAF               False   
                                        FSNTOA_CERES                    False   
                                        FSNTOA_ERBE                     False   
                                        FSNTOAC_CERES-EBAF              False   
                                        FSNTOAC_CERES                   False   
                                        FSNTOAC_ERBE                    False   
                                        FSNT_CAM                        False   
                                        LHFLX_JRA25                     False   
                                        LHFLX_ERA40                     False   
                                        LHFLX_WHOI                      False   
...                                                                       ...   
                                        PRECT_GPCP                      False   
                                        PREH2O_NVAP                     False   
                                        PREH2O_AIRS                     False   
                                        PREH2O_JRA25                    False   
                                        PREH2O_ERAI                     False   
                                        PREH2O_ERA40                    False   
                                        PSL_JRA25                       False   
                                        PSL_ERAI                        False   
                                        SHFLX_JRA25                     False   
                                        SHFLX_NCEP                      False   
                                        SHFLX_LARYEA                    False   
                                        STRESS_MAG_ERS                  False   
                                        STRESS_MAG_LARYEA               False   
                                        STRESS_MAG_JRA25                False   
                                        SWCF_CERES-EBAF                 False   
                                        SWCF_CERES                      False   
                                        SWCF_ERBE                       False   
                       

That worked, you can see that the column *Description* is empty and that all flags are set `False`. Here in the default reading function the flag is set True, if the ``var_info_dict`` is provided and information for a given variable is available. We illustrate that in the following, our second example, where we load the corrected ascii file and provide the info dictionary that we imported from the Excel sheet. Furthermore, we use the optional input parameter `run_id` to shorten to shorten the width of the HTML table display of the dataframe.

In [10]:
df1 = read_file_custom(case_ok, var_info_dict, run_id="Run1", verbose=False)
df1

Flag     Model  \
Run  Years   Variable           Description                            
Run1 150-180 RESTOM             TOmodel net flux      True     0.020   
             RESSURF            SRF net flux          True     0.027   
             RESTOA_CERES-EBAF  TOA  net flux         True     2.109   
             RESTOA_ERBE                             False     2.109   
             SOLIN_CERES-EBAF                        False   340.200   
             SOLIN_CERES                             False   340.200   
             CLDTOT_ISCCP       Total cloud cover     True    70.746   
             CLDTOT_CLOUDSAT                         False    70.746   
             FLDS_ISCCP         LW down SRF           True   347.663   
             FLNS_ISCCP         LW net SRF            True    56.374   
             FLUT_CERES-EBAF    LW up Top             True   238.633   
             FLUT_CERES                              False   238.633   
             FLUT_ERBE                               False   238.633   
             FLUTC_CERES-EBAF   LW up Top Clearsky    True   264.795   
             FLUTC_CERES                             False   264.795   
             FLUTC_ERBE                              False   264.795   
             FLNT_CAM                                False   237.303   
             FSDS_ISCCP         SW down SRF           True   187.745   
             FSNS_ISCCP         SW net SRF            True   163.761   
             FSNS_LARYEA                             False   163.761   
             FSNTOA_CERES-EBAF                       False   240.742   
             FSNTOA_CERES       SW net TOA            True   240.742   
             FSNTOA_ERBE                             False   240.742   
             FSNTOAC_CERES-EBAF                      False   289.674   
             FSNTOAC_CERES      SW net TOA clearsky   True   289.674   
             FSNTOAC_ERBE                            False   289.674   
             FSNT_CAM                                False   237.323   
             LHFLX_JRA25        Lat Heat Flux         True    85.136   
             LHFLX_ERA40                             False    85.136   
             LHFLX_WHOI                              False    85.136   
...                                                    ...       ...   
             PRECT_GPCP         Precipitation         True     2.914   
             PREH2O_NVAP                             False    25.663   
             PREH2O_AIRS                             False    25.663   
             PREH2O_JRA25                            False    25.663   
             PREH2O_ERAI                             False    25.663   
             PREH2O_ERA40       Precipitable water    True    25.663   
             PSL_JRA25          SeaLev pressure       True  1011.611   
             PSL_ERAI                                False  1011.611   
             SHFLX_JRA25        Sens Heat Flux        True    22.225   
             SHFLX_NCEP                              False    22.225   
             SHFLX_LARYEA                            False    22.225   
             STRESS_MAG_ERS                          False     0.063   
             STRESS_MAG_LARYEA                       False     0.063   
             STRESS_MAG_JRA25                        False     0.063   
             SWCF_CERES-EBAF    SW Cloud Forc         True   -48.932   
             SWCF_CERES                              False   -48.932   
             SWCF_ERBE                               False   -48.932   
             AODVIS                                  False       NaN   
             AODDUST                                 False       NaN   
             SST_HADISST                             False    21.370   
             SST_HADISST_PI     SST preindustrial     True    21.690   
             SST_HADISST_PD                          False    21.359   
             TREFHT_LEGATES                          False   287.917   
             TREFHT_JR

As specified in the docstring of the reading method, the actual test case ID is stored (as attribute `test_case` in the Dataframe) in form of a dictionary that maps test_case with specified run_id:

In [11]:
print(df1.test_case)

N1850_f19_tn14_r265_ctrl_20180411


Reading worked, check, if both dataframes have the same dimension.

In [12]:
print(df.shape)
print(df1.shape)

(63, 5)
(63, 5)


You can change the appearance of the way a DataFrame is displayed. For instance ...

In [13]:
df1.style.background_gradient(cmap="GnBu", low=0.5, high=0.5, axis=0).highlight_null("red")

... applies a column based colour gradient based on the cell values in each column (similar to Excel feature *Conditional formatting*). Here, we provided a colormap of our choice (see [here](https://matplotlib.org/examples/color/colormaps_reference.html) for options). The input parameters `low` and `high` are optional, and can be interpreted as an percentage specification of the colour range used from the colourbar for the display (i.e. here, we use 0.5, i.e. 50% both for the lower and upper end).

[Follow this link](https://pandas.pydata.org/pandas-docs/stable/style.html) for more details related to pandas styling. 

### Importing multiple result files and concatenating them into one Dataframe

Now we have a method that can import the results from a single run into a datframe that can be used for further analysis. In the following, we basically do this for all available files and put the results into one big `Dataframe`. 

To do this, a custom method `read_and_merge_all` was defined in [helper_funcs.py](https://github.com/jgliss/my_py3_scripts/blob/master/notebooks/helper_funcs.py). 

The following cells show, how this method may be used to either keep the original test case IDs as index or to replace them with a shorter version. 

In [14]:
from helper_funcs import read_and_merge_all
merged = read_and_merge_all(file_list=files, var_info_dict=var_info_dict)
merged

Flag  \
Run                               Years  Variable           Description                  
N1850C53CLM45L32_f09_tn11_191017  71-100 RESTOM             TOmodel net flux      True   
                                         RESSURF            SRF net flux          True   
                                         RESTOA_CERES-EBAF  TOA  net flux         True   
                                         RESTOA_ERBE                             False   
                                         SOLIN_CERES-EBAF                        False   
                                         SOLIN_CERES                             False   
                                         CLDTOT_ISCCP       Total cloud cover     True   
                                         CLDTOT_CLOUDSAT                         False   
                                         FLDS_ISCCP         LW down SRF           True   
                                         FLNS_ISCCP         LW net SRF            True   
                                         FLUT_CERES-EBAF    LW up Top             True   
                                         FLUT_CERES                              False   
                                         FLUT_ERBE                               False   
                                         FLUTC_CERES-EBAF   LW up Top Clearsky    True   
                                         FLUTC_CERES                             False   
                                         FLUTC_ERBE                              False   
                                         FLNT_CAM                                False   
                                         FSDS_ISCCP         SW down SRF           True   
                                         FSNS_ISCCP         SW net SRF            True   
                                         FSNS_LARYEA                             False   
                                         FSNTOA_CERES-EBAF                       False   
                                         FSNTOA_CERES       SW net TOA            True   
                                         FSNTOA_ERBE                             False   
                                         FSNTOAC_CERES-EBAF                      False   
                                         FSNTOAC_CERES      SW net TOA clearsky   True   
                                         FSNTOAC_ERBE                            False   
                                         FSNT_CAM                                False   
                                         LHFLX_JRA25        Lat Heat Flux         True   
                                         LHFLX_ERA40                             False   
                                         LHFLX_WHOI                              False   
...                                                                                ...   
N1850_f19_tn14_r265_ctrl_20180411 90-120 PRECT_GPCP         Precipitation         True   
                                         PREH2O_NVAP                             False   
                                         PREH2O_AIRS                             False   
                                         PREH2O_JRA25                            False   
                                         PREH2O_ERAI                             False   
                                         PREH2O_ERA40       Precipitable water    True   
                                         PSL_JRA25          SeaLev pressure       True   
                                         PSL_ERAI                                False   
                                         SHFLX_JRA25        Sens Heat Flux        True   
                                         SHFLX_NCEP                              False   
                                         SHFLX_LARYEA                            False   
                                         STRESS_MAG_ERS                          False   
                                

This is rather unhandy, since the names of the run IDs are rather long. This can be changed directly when loading the Dataframe:

In [15]:
merged = read_and_merge_all(file_list=files, var_info_dict=var_info_dict, replace_runid_prefix="Run #")
merged

Flag     Model  \
Run    Years  Variable           Description                            
Run #1 71-100 RESTOM             TOmodel net flux      True    -0.489   
              RESSURF            SRF net flux          True    -0.489   
              RESTOA_CERES-EBAF  TOA  net flux         True     1.529   
              RESTOA_ERBE                             False     1.529   
              SOLIN_CERES-EBAF                        False   340.206   
              SOLIN_CERES                             False   340.206   
              CLDTOT_ISCCP       Total cloud cover     True    63.621   
              CLDTOT_CLOUDSAT                         False    63.621   
              FLDS_ISCCP         LW down SRF           True   338.280   
              FLNS_ISCCP         LW net SRF            True    55.819   
              FLUT_CERES-EBAF    LW up Top             True   238.148   
              FLUT_CERES                              False   238.148   
              FLUT_ERBE                               False   238.148   
              FLUTC_CERES-EBAF   LW up Top Clearsky    True   261.783   
              FLUTC_CERES                             False   261.783   
              FLUTC_ERBE                              False   261.783   
              FLNT_CAM                                False   236.838   
              FSDS_ISCCP         SW down SRF           True   187.801   
              FSNS_ISCCP         SW net SRF            True   163.679   
              FSNS_LARYEA                             False   163.679   
              FSNTOA_CERES-EBAF                       False   239.677   
              FSNTOA_CERES       SW net TOA            True   239.677   
              FSNTOA_ERBE                             False   239.677   
              FSNTOAC_CERES-EBAF                      False   287.999   
              FSNTOAC_CERES      SW net TOA clearsky   True   287.999   
              FSNTOAC_ERBE                            False   287.999   
              FSNT_CAM                                False   236.348   
              LHFLX_JRA25        Lat Heat Flux         True    87.904   
              LHFLX_ERA40                             False    87.904   
              LHFLX_WHOI                              False    87.904   
...                                                     ...       ...   
Run #4 90-120 PRECT_GPCP         Precipitation         True     2.918   
              PREH2O_NVAP                             False    25.733   
              PREH2O_AIRS                             False    25.733   
              PREH2O_JRA25                            False    25.733   
              PREH2O_ERAI                             False    25.733   
              PREH2O_ERA40       Precipitable water    True    25.733   
              PSL_JRA25          SeaLev pressure       True  1011.614   
              PSL_ERAI                                False  1011.614   
              SHFLX_JRA25        Sens Heat Flux        True    22.229   
              SHFLX_NCEP                              False    22.229   
              SHFLX_LARYEA                            False    22.229   
              STRESS_MAG_ERS                          False     0.063   
              STRESS_MAG_LARYEA                       False     0.063   
              STRESS_MAG_JRA25                        False     0.063   
              SWCF_CERES-EBAF    SW Cloud Forc         True   -48.959   
              SWCF_CERES                              False   -48.959   
              SWCF_ERBE                               False   -48.959   
              AODVIS                                  False       NaN   
              AODDUST                                 False       NaN   
              SST_HADISST                             False    21.405   
              SST_HADISST_PI     SST preindustrial     True    21.726   
              SST_HADISST_PD                          False    21.396   
              TREFHT_LEGATES            

Call:

In [16]:
merged.test_case

Run #1     N1850C53CLM45L32_f09_tn11_191017
Run #2                N1850_f09_tn14_230218
Run #3             N1850_f19_tn14_r227_ctrl
Run #4    N1850_f19_tn14_r265_ctrl_20180411
dtype: object

to get the corresponding test_case IDs.

### Rearranging and restructuring of the imported data

Now we can use the `Flag` column to select only the variables that we are interested in. We then use the `drop` method to remove the column `Flag` since we don't need it anymore.

In [17]:
flagged = merged[merged.Flag == True].drop("Flag", axis=1)
flagged.shape
print(len(flagged.index.levels[2]), len(merged.index.levels[2]))
merged[merged.Flag].index.get_level_values("Variable").unique().values

63 63


array(['RESTOM', 'RESSURF', 'RESTOA_CERES-EBAF', 'CLDTOT_ISCCP',
       'FLDS_ISCCP', 'FLNS_ISCCP', 'FLUT_CERES-EBAF', 'FLUTC_CERES-EBAF',
       'FSDS_ISCCP', 'FSNS_ISCCP', 'FSNTOA_CERES', 'FSNTOAC_CERES',
       'LHFLX_JRA25', 'LWCF_CERES-EBAF', 'PRECT_GPCP', 'PREH2O_ERA40',
       'PSL_JRA25', 'SHFLX_JRA25', 'SWCF_CERES-EBAF', 'SST_HADISST_PI',
       'TREFHT_JRA25', 'TS_NCEP', 'TS_LAND_NCEP', 'U_200_JRA25',
       'Z3_500_JRA25'], dtype=object)

In the following cell, you can interacively select which Variables you wish to keep for further analysis.

In [18]:
import ipywidgets as ipw
from IPython import display
from copy import deepcopy
    
class SelectVariable():
    output = ipw.Output()
    def __init__(self, df):
        self.df = deepcopy(df)
        self.vals = tuple(self.df.index.levels[2].values)
        
        self.df_edit = self.df
        
        self.init_layout()
        self.init_widgets()
        self.init_actions()
        self.init_display()
        self.select_flagged()
        self.print_current(1)
        
    def init_widgets(self):
        
        self.btn_unselect_all = ipw.Button(description='Unselect all')
        self.btn_select_all = ipw.Button(description='Select all')
        self.btn_flagged = ipw.Button(description="Flagged")
        self.btn_apply = ipw.Button(description='Apply')
        self.btn_apply.style.button_color = "lime"

        self.var_selector = ipw.SelectMultiple(description="Variables", 
                                               options=self.vals, 
                                               value=self.vals, 
                                               min_width='150px',
                                               layout=self.box_layout)
        
        self.current_disp = ipw.Textarea(value='',
                                         description='Current:',
                                         disabled=True,
                                         layout=self.box_layout)
        #self.output = ipw.Output()
        
    def init_actions(self):
        #what happens when the state of the selection is changed (display current selection)
        self.var_selector.observe(self.print_current)
        #what happens when buttons are clicked
        self.btn_select_all.on_click(self.on_select_all_vars_clicked)
        self.btn_unselect_all.on_click(self.on_unselect_all_vars_clicked)
        self.btn_flagged.on_click(self.on_flagged_clicked)
        self.btn_apply.on_click(self.on_click_apply)
        
    def init_layout(self):
        self.box_layout = ipw.Layout(flex='0 1 auto', height='250px', min_height='150px', width='auto')
        self.disp_layout = ipw.Layout(flex='0 1 auto', height='250px', min_height='150px', width='auto')
    
    def init_display(self):
        self.btns = ipw.VBox([self.btn_select_all, 
                              self.btn_unselect_all,
                              self.btn_flagged,
                              ipw.Label(),
                              self.btn_apply])
    
        self.edit_area = ipw.HBox([self.var_selector, 
                                   self.current_disp, 
                                   self.btns])
        
        self.layout = ipw.VBox([self.edit_area, self.output])
    
    def on_unselect_all_vars_clicked(self, b):
        self.unselect_all()
    
    def on_select_all_vars_clicked(self, b):
        self.select_all()
    
    def on_flagged_clicked(self, b):
        self.select_flagged()
        
    def unselect_all(self):
        self.var_selector.value = ()
    
    def select_all(self):
        self.var_selector.value = self.var_selector.options
    
    def select_flagged(self):
        self.var_selector.value = list(self.df[self.df.Flag].index.get_level_values("Variable").unique().values)
        
    def on_click_apply(self, b):
        idx = pd.IndexSlice
        self.df_edit = self.df.loc[idx[:, :, self.var_selector.value, :], :]
        self.output.clear_output()
        self.output.append_display_data(self.df_edit.head())
        self.output

    def print_current(self, b):
        s=""
        for item in self.var_selector.value:
            s += "{}\n".format(item)
        self.current_disp.value = s
        
    def __repr__(self):
        return self.layout
    
    def __call__(self):
        return self.layout
    
selector = SelectVariable(df=merged)
#show
selector()

Now access the current selection and continue.

In [19]:
selection  = selector.df_edit

For visualisation this display requires a lot of scrolling. We can make the table `wider` by unstacking certain indices, e.g. the two outermost indices `Run` and `Years`.

In [20]:
selection_unstacked = selection.unstack(["Run", "Years"])
selection_unstacked

Flag                                \
Run                                    Run #1 Run #2  Run #3                  
Years                                  71-100   1-20 185-215 310-340 80-110   
Variable           Description                                                
AODDUST                                 False  False   False   False  False   
AODVIS                                  False  False   False   False  False   
CLDTOT_CLOUDSAT                         False  False   False   False  False   
CLDTOT_ISCCP       Total cloud cover     True   True    True    True   True   
FLDS_ISCCP         LW down SRF           True   True    True    True   True   
FLNS_ISCCP         LW net SRF            True   True    True    True   True   
FLNT_CAM                                False  False   False   False  False   
FLUTC_CERES                             False  False   False   False  False   
FLUTC_CERES-EBAF   LW up Top Clearsky    True   True    True    True   True   
FLUTC_ERBE                              False  False   False   False  False   
FLUT_CERES                              False  False   False   False  False   
FLUT_CERES-EBAF    LW up Top             True   True    True    True   True   
FLUT_ERBE                               False  False   False   False  False   
FSDS_ISCCP         SW down SRF           True   True    True    True   True   
FSNS_ISCCP         SW net SRF            True   True    True    True   True   
FSNS_LARYEA                             False  False   False   False  False   
FSNTOAC_CERES      SW net TOA clearsky   True   True    True    True   True   
FSNTOAC_CERES-EBAF                      False  False   False   False  False   
FSNTOAC_ERBE                            False  False   False   False  False   
FSNTOA_CERES       SW net TOA            True   True    True    True   True   
FSNTOA_CERES-EBAF                       False  False   False   False  False   
FSNTOA_ERBE                             False  False   False   False  False   
FSNT_CAM                                False  False   False   False  False   
LHFLX_ERA40                             False  False   False   False  False   
LHFLX_JRA25        Lat Heat Flux         True   True    True    True   True   
LHFLX_WHOI                              False  False   False   False  False   
LWCF_CERES                              False  False   False   False  False   
LWCF_CERES-EBAF    LW Cloud Forc         True   True    True    True   True   
LWCF_ERBE                               False  False   False   False  False   
PRECT_GPCP         Precipitation         True   True    True    True   True   
...                                       ...    ...     ...     ...    ...   
PREH2O_JRA25                            False  False   False   False  False   
PREH2O_NVAP                             False  False   False   False  False   
PSL_ERAI                                False  False   False   False  False   
PSL_JRA25          SeaLev pressure       True   True    True    True   True   
RESSURF            SRF net flux          True   True    True    True   True   
RESTOA_CERES-EBAF  TOA  net flux         True   True    True    True   True   
RESTOA_ERBE                             False  False   False   False  False   
RESTOM             TOmodel net flux      True   True    True    True   True   
SHFLX_JRA25        Sens Heat Flux        True   True    True    True   True   
SHFLX_LARYEA                            False  False   False   False  False   
SHFLX_NCEP                              False  False   False   False  False   
SOLIN_CERES                             False  False   False   False  False   
SOLIN_CERES-EBAF                        False  False   False   False  False   
SST_HADISST                             False  False   False   False  False   
SST_HADISST_PD                          False  False   False   False  False   
SST_HADISST_PI     SST preindustrial     True   True    True    True   True   
STRESS_MAG_E

Well, this is better but also not extremely illustrative / intuitive. It becomes more intuitive if we just look at one parameter that we are interested in (e.g. RMSE). 

#### Extracting the Bias of each model run relative to the observations

Retrieving a table that illustrates the Bias of each run for each flagged variable is straight forward. We just extract the `Bias` column from our flagged frame:

In [21]:
bias_table = selection_unstacked["Bias"]
bias_table.head()

Run                               Run #1 Run #2  Run #3                Run #4
Years                             71-100   1-20 185-215 310-340 80-110 90-120
Variable        Description                                                  
AODDUST                              NaN    NaN     NaN     NaN    NaN    NaN
AODVIS                               NaN    NaN     NaN     NaN    NaN    NaN
CLDTOT_CLOUDSAT                   -3.203  1.762   1.720   1.411  2.133  3.923
CLDTOT_ISCCP    Total cloud cover -3.179  1.786   1.744   1.435  2.157  3.947
FLDS_ISCCP      LW down SRF       -5.066 -1.799  10.514  11.499  5.162  4.507

#### Computing RMSE relative error

In the following we extract the subset containing the *RSME* information of the flagged variables for all runs in order to compute the relative error for each run based on the average *RMSE* of all runs:

$$\frac{RMSE_{Run}\,-\,\overline{RMSE_{All\,Runs}}}{\overline{RMSE_{All\,Runs}}}$$


In [22]:
rmse = selection_unstacked["RMSE"]
rmse

Run                                     Run #1  Run #2  Run #3          \
Years                                   71-100    1-20 185-215 310-340   
Variable           Description                                           
AODDUST                                    NaN     NaN     NaN     NaN   
AODVIS                                     NaN     NaN     NaN     NaN   
CLDTOT_CLOUDSAT                          9.731  10.382  10.886  10.952   
CLDTOT_ISCCP       Total cloud cover    11.323  11.881  12.992  13.078   
FLDS_ISCCP         LW down SRF          14.450  15.351  16.891  17.664   
FLNS_ISCCP         LW net SRF           11.967  14.953  14.098  14.174   
FLNT_CAM                                   NaN     NaN     NaN     NaN   
FLUTC_CERES                              8.384   7.017   5.873   5.884   
FLUTC_CERES-EBAF   LW up Top Clearsky    6.042   4.778   4.662   4.738   
FLUTC_ERBE                               5.725   5.164   5.765   5.968   
FLUT_CERES                               6.770   6.407   7.388   7.743   
FLUT_CERES-EBAF    LW up Top             6.855   6.169   7.188   7.467   
FLUT_ERBE                                7.887   9.451  10.546  10.989   
FSDS_ISCCP         SW down SRF          13.380  15.089  15.915  16.082   
FSNS_ISCCP         SW net SRF           12.711  12.632  13.587  13.727   
FSNS_LARYEA                             36.124  39.891  39.707  39.902   
FSNTOAC_CERES      SW net TOA clearsky  18.458  17.432  15.977  15.710   
FSNTOAC_CERES-EBAF                       7.845   8.282   6.661   6.781   
FSNTOAC_ERBE                            15.441  13.441  14.489  14.214   
FSNTOA_CERES       SW net TOA           12.307  10.795  12.096  12.125   
FSNTOA_CERES-EBAF                       10.530  10.839  11.822  11.968   
FSNTOA_ERBE                             12.491  14.908  15.645  15.914   
FSNT_CAM                                   NaN     NaN     NaN     NaN   
LHFLX_ERA40                             19.019  15.466  17.077  17.587   
LHFLX_JRA25        Lat Heat Flux        17.176  14.578  14.947  15.116   
LHFLX_WHOI                              30.655  26.065  27.780  28.496   
LWCF_CERES                               7.158   6.792   6.663   6.782   
LWCF_CERES-EBAF    LW Cloud Forc         6.156   5.683   5.860   5.980   
LWCF_ERBE                                9.564   9.126   8.595   8.707   
PRECT_GPCP         Precipitation         1.162   0.891   1.043   1.069   
...                                        ...     ...     ...     ...   
PREH2O_JRA25                             4.396   4.976   8.887   9.200   
PREH2O_NVAP                              2.406   2.422   4.824   5.113   
PSL_ERAI                                 1.350   1.634   1.917   2.040   
PSL_JRA25          SeaLev pressure       1.333   1.512   1.825   1.939   
RESSURF            SRF net flux            NaN     NaN     NaN     NaN   
RESTOA_CERES-EBAF  TOA  net flux         8.842   8.358   9.011   8.937   
RESTOA_ERBE                              8.992   8.706   8.977   8.906   
RESTOM             TOmodel net flux        NaN     NaN     NaN     NaN   
SHFLX_JRA25        Sens Heat Flux       11.761  11.926  11.051  11.138   
SHFLX_LARYEA                            12.673  13.511  12.022  12.050   
SHFLX_NCEP                              16.446  15.897  14.937  14.929   
SOLIN_CERES                              1.226   1.289   1.296   1.296   
SOLIN_CERES-EBAF                         0.167   0.403   0.417   0.417   
SST_HADISST                              1.129   0.967   2.103   2.238   
SST_HADISST_PD                           1.231   1.002   2.031   2.166   
SST_HADISST_PI     SST preindustrial     1.059   1.223   2.458   2.606   
STRESS_MAG_ERS                           0.019   0.019   0.020   0.021   
STRESS_MAG_JRA25                         0.021   0.020   0.022   0.022   
STRESS_MAG_LARYEA                        0.017   0.018   0.022   0.022   
SWCF_CERES                              14.396  13.657  14.795  14.720   
SWCF

In [23]:
rmse_mean = rmse.mean(axis=1, skipna=True)
rmse_mean

Variable            Description        
AODDUST                                          NaN
AODVIS                                           NaN
CLDTOT_CLOUDSAT                            10.567667
CLDTOT_ISCCP        Total cloud cover      12.438000
FLDS_ISCCP          LW down SRF            16.059000
FLNS_ISCCP          LW net SRF             13.949333
FLNT_CAM                                         NaN
FLUTC_CERES                                 6.892000
FLUTC_CERES-EBAF    LW up Top Clearsky      5.111333
FLUTC_ERBE                                  5.504333
FLUT_CERES                                  7.057167
FLUT_CERES-EBAF     LW up Top               6.962667
FLUT_ERBE                                   9.445833
FSDS_ISCCP          SW down SRF            15.322500
FSNS_ISCCP          SW net SRF             13.238333
FSNS_LARYEA                                38.799000
FSNTOAC_CERES       SW net TOA clearsky    17.282000
FSNTOAC_CERES-EBAF                          7.605833
FSNTOA

The next step is (semi) straight forward (we have to use the `div` and `subtract` methods of the Dataframe rather than `/` and `-` operators in order to specify that we want to apply them in the horizontal and not in the vertical direction.

In [24]:
typical_rmse = rmse.subtract(rmse_mean, axis=0).div(rmse_mean, axis=0)
typical_rmse

Run                                       Run #1    Run #2    Run #3  \
Years                                     71-100      1-20   185-215   
Variable           Description                                         
AODDUST                                      NaN       NaN       NaN   
AODVIS                                       NaN       NaN       NaN   
CLDTOT_CLOUDSAT                        -0.079172 -0.017569  0.030123   
CLDTOT_ISCCP       Total cloud cover   -0.089645 -0.044782  0.044541   
FLDS_ISCCP         LW down SRF         -0.100193 -0.044087  0.051809   
FLNS_ISCCP         LW net SRF          -0.142110  0.071951  0.010658   
FLNT_CAM                                     NaN       NaN       NaN   
FLUTC_CERES                             0.216483  0.018137 -0.147853   
FLUTC_CERES-EBAF   LW up Top Clearsky   0.182079 -0.065215 -0.087909   
FLUTC_ERBE                              0.040090 -0.061830  0.047357   
FLUT_CERES                             -0.040691 -0.092129  0.046879   
FLUT_CERES-EBAF    LW up Top           -0.015463 -0.113989  0.032363   
FLUT_ERBE                              -0.165029  0.000547  0.116471   
FSDS_ISCCP         SW down SRF         -0.126774 -0.015239  0.038669   
FSNS_ISCCP         SW net SRF          -0.039834 -0.045801  0.026338   
FSNS_LARYEA                            -0.068945  0.028145  0.023403   
FSNTOAC_CERES      SW net TOA clearsky  0.068048  0.008680 -0.075512   
FSNTOAC_CERES-EBAF                      0.031445  0.088901 -0.124225   
FSNTOAC_ERBE                            0.062296 -0.075298 -0.003199   
FSNTOA_CERES       SW net TOA           0.020650 -0.104744  0.003151   
FSNTOA_CERES-EBAF                      -0.074394 -0.047233  0.039175   
FSNTOA_ERBE                            -0.136079  0.031089  0.082062   
FSNT_CAM                                     NaN       NaN       NaN   
LHFLX_ERA40                             0.118863 -0.090155  0.004618   
LHFLX_JRA25        Lat Heat Flux        0.113433 -0.054982 -0.031062   
LHFLX_WHOI                              0.112482 -0.054091  0.008147   
LWCF_CERES                              0.052389 -0.001421 -0.020387   
LWCF_CERES-EBAF    LW Cloud Forc        0.044275 -0.035963 -0.005937   
LWCF_ERBE                               0.093133  0.043071 -0.017621   
PRECT_GPCP         Precipitation        0.125424 -0.137046  0.010169   
...                                          ...       ...       ...   
PREH2O_JRA25                           -0.363989 -0.280075  0.285766   
PREH2O_NVAP                            -0.335298 -0.330878  0.332719   
PSL_ERAI                               -0.241857 -0.082366  0.076563   
PSL_JRA25          SeaLev pressure     -0.219250 -0.114408  0.068918   
RESSURF            SRF net flux              NaN       NaN       NaN   
RESTOA_CERES-EBAF  TOA  net flux       -0.014764 -0.068695  0.004067   
RESTOA_ERBE                             0.007827 -0.024228  0.006146   
RESTOM             TOmodel net flux          NaN       NaN       NaN   
SHFLX_JRA25        Sens Heat Flux       0.028224  0.042650 -0.033849   
SHFLX_LARYEA                            0.011991  0.078909 -0.039994   
SHFLX_NCEP                              0.070715  0.034972 -0.027529   
SOLIN_CERES                            -0.044551  0.004546  0.010001   
SOLIN_CERES-EBAF                       -0.552279  0.080429  0.117962   
SST_HADISST                            -0.297813 -0.398570  0.307971   
SST_HADISST_PD                         -0.230465 -0.373620  0.269640   
SST_HADISST_PI     SST preindustrial   -0.433336 -0.345581  0.315259   
STRESS_MAG_ERS                         -0.025641 -0.025641  0.025641   
STRESS_MAG_JRA25                       -0.015625 -0.062500  0.031250   
STRESS_MAG_LARYEA                      -0.142857 -0.092437  0.109244   
SWCF_CERES                             -0.020525 -0.070805  0.006622   
SWCF_CERES-EBAF    SW Cloud Forc        0.022955 -0.122700  0.035142   
SWCF_ERBE                              -

If we want, we can now add the typical RMSE to our original dataframe (containing the only flagged data, since it was computed from this). First we have to stack it:

In [25]:
stacked = typical_rmse.stack(level=(0,1)).reorder_levels(order=(2,3,0,1))
stacked.head()

Run     Years    Variable         Description
Run #1  71-100   CLDTOT_CLOUDSAT                -0.079172
Run #2  1-20     CLDTOT_CLOUDSAT                -0.017569
Run #3  185-215  CLDTOT_CLOUDSAT                 0.030123
        310-340  CLDTOT_CLOUDSAT                 0.036369
        80-110   CLDTOT_CLOUDSAT                 0.030880
dtype: float64

In [26]:
selection["RMSE_typical"] = stacked
selection

Flag     Model  \
Run    Years  Variable           Description                            
Run #1 71-100 RESTOM             TOmodel net flux      True    -0.489   
              RESSURF            SRF net flux          True    -0.489   
              RESTOA_CERES-EBAF  TOA  net flux         True     1.529   
              RESTOA_ERBE                             False     1.529   
              SOLIN_CERES-EBAF                        False   340.206   
              SOLIN_CERES                             False   340.206   
              CLDTOT_ISCCP       Total cloud cover     True    63.621   
              CLDTOT_CLOUDSAT                         False    63.621   
              FLDS_ISCCP         LW down SRF           True   338.280   
              FLNS_ISCCP         LW net SRF            True    55.819   
              FLUT_CERES-EBAF    LW up Top             True   238.148   
              FLUT_CERES                              False   238.148   
              FLUT_ERBE                               False   238.148   
              FLUTC_CERES-EBAF   LW up Top Clearsky    True   261.783   
              FLUTC_CERES                             False   261.783   
              FLUTC_ERBE                              False   261.783   
              FLNT_CAM                                False   236.838   
              FSDS_ISCCP         SW down SRF           True   187.801   
              FSNS_ISCCP         SW net SRF            True   163.679   
              FSNS_LARYEA                             False   163.679   
              FSNTOA_CERES-EBAF                       False   239.677   
              FSNTOA_CERES       SW net TOA            True   239.677   
              FSNTOA_ERBE                             False   239.677   
              FSNTOAC_CERES-EBAF                      False   287.999   
              FSNTOAC_CERES      SW net TOA clearsky   True   287.999   
              FSNTOAC_ERBE                            False   287.999   
              FSNT_CAM                                False   236.348   
              LHFLX_JRA25        Lat Heat Flux         True    87.904   
              LHFLX_ERA40                             False    87.904   
              LHFLX_WHOI                              False    87.904   
...                                                     ...       ...   
Run #4 90-120 PRECT_GPCP         Precipitation         True     2.918   
              PREH2O_NVAP                             False    25.733   
              PREH2O_AIRS                             False    25.733   
              PREH2O_JRA25                            False    25.733   
              PREH2O_ERAI                             False    25.733   
              PREH2O_ERA40       Precipitable water    True    25.733   
              PSL_JRA25          SeaLev pressure       True  1011.614   
              PSL_ERAI                                False  1011.614   
              SHFLX_JRA25        Sens Heat Flux        True    22.229   
              SHFLX_NCEP                              False    22.229   
              SHFLX_LARYEA                            False    22.229   
              STRESS_MAG_ERS                          False     0.063   
              STRESS_MAG_LARYEA                       False     0.063   
              STRESS_MAG_JRA25                        False     0.063   
              SWCF_CERES-EBAF    SW Cloud Forc         True   -48.959   
              SWCF_CERES                              False   -48.959   
              SWCF_ERBE                               False   -48.959   
              AODVIS                                  False       NaN   
              AODDUST                                 False       NaN   
              SST_HADISST                             False    21.405   
              SST_HADISST_PI     SST preindustrial     True    21.726   
              SST_HADISST_PD                          False    21.396   
              TREFHT_LEGATES            

### Conditional formatting of tables (Dataframes)

This section illustrates, how we can perform conditional formatting of the color tables. As discussed above, we can apply background colour gradients to the data. In the example above we had a multiindex data type specifying model run, year-range and variable in stacked format (long table) and the four data columns specifying results from model and observation as well as bias and RMSE. 

Now, in the following we illustrate how we can apply this colour highlighting for the two unstacked tables that we just created and that contain Bias and relative error. 

Starting with the Bias data, we show an example that does not work for our purposes (since it only allows for conditional formatting of either rows or columns.

#### NOT how we want it (using the style method `background_gradient`)

The most straight forward example for conditional formatting of a Dataframe is shown in the following. In the example we use the `Bias` table and, similar to the example above, apply a value based colormap. Here, we use a *diverging colormap (bwr)* which has white as center color. Like in the example above, we use the style method `background_gradient` which can perform the formatting either in a **rowwise** or **columnwise** manner (using input argument `axis=1` or `axis=0`, respectively). 

Note, however, that this is not what we are aiming for in this example, rather, we want the colour formatting to be applied based on the values available the **whole table** and not individually for **columns** or **rows** (which is done in the next section). Nonetheless, in the cell below we show what we get if we use the method `backgroun_gradient`.  

Again, we use the `low` and `high` parameters to specify the colorrange that we use to map the values (see above).

In [27]:
bias_table.style.background_gradient(cmap="bwr", low=0.5, high=0.5, axis=1).highlight_null("white")

Now, this worked nicely but there are mainly two problems with this representation:

1. As mentioned above, one problem here is that the colour coding can only be performed row or column wise using the input parameter `axis` (and not based on the values of the whole table, see [here](https://pandas.pydata.org/pandas-docs/stable/style.html#Building-Styles-Summary) for details)
2. If we use the symmetric colormap as is (i.e. center colour is white), then, the color white will be mapped to the midpoint value of the considered value range (e.g. min=-2, max=4 => (4 - -2)/2 = 3 => 1 == white). However, what we want is a *shifter diverging colormap* that ensures that the value 0 is mapped white, even if min != -max.
3. Further, we might wish to have control over the number of significant digits that are displayed in the table

All these problems will be solved in the following.

#### How we want it (using `apply` style method and custom colormapping)

In the following, a custom method is defined that performs the colour formatting considering all rows and columns at the same time and furthermore using a diverging colour map that is dynamically shifted such that value 0 is white (method `shifted_color_map` in [helper_funcs.py](https://github.com/jgliss/my_py3_scripts/blob/master/notebooks/helper_funcs.py)) also if `-vmin != vmax` (like usually).

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from helper_funcs import shifted_color_map

def background_gradient_list(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    low = m - (rng * low)
    high = M + (rng * high)
    cm = shifted_color_map(vmin=low, vmax=high, cmap=cmap)
    norm = colors.Normalize(low, high)
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in cm(normed)]
    return ['background-color: %s' % color for color in c]

bias_table.style.apply(background_gradient_list,
                       cmap='bwr',
                       m=bias_table.min().min(),
                       M=bias_table.max().max(),low=0.5, high=0.5).highlight_null("white").format("{:.2f}")